In [24]:
import os
import alpha_vantage

In [25]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='D:\Real Time Finance API Client\apikey.env')  
api_key = os.getenv("ALPHAVANTAGE_API_KEY")

print("API Key:", api_key)


API Key: JHXP8S1TOLVS5YLO


In [29]:
import requests
import time

base_url = "https://www.alphavantage.co/query"
stocks = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA"]
all_data = {}

for symbol in stocks:
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": symbol,
        "outputsize": "compact",
        "apikey": api_key
    }
    
    response = requests.get(base_url, params=params)
    data = response.json()
    all_data[symbol] = data
    
    print(f"Fetched data for {symbol}")
    
    # Add delay to respect API rate limits (Alpha Vantage free tier: 5 calls/minute)
    time.sleep(12)  # 12 seconds between calls = 5 calls per minute

print("\nAll data collected!")
# Access individual stock data like: all_data["AAPL"]

Fetched data for AAPL
Fetched data for GOOGL
Fetched data for MSFT
Fetched data for AMZN
Fetched data for TSLA

All data collected!


In [30]:
data

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'TSLA',
  '3. Last Refreshed': '2025-08-01',
  '4. Output Size': 'Compact',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2025-08-01': {'1. open': '306.2050',
   '2. high': '309.3100',
   '3. low': '297.8200',
   '4. close': '302.6300',
   '5. volume': '89121446'},
  '2025-07-31': {'1. open': '319.6050',
   '2. high': '321.3700',
   '3. low': '306.1000',
   '4. close': '308.2700',
   '5. volume': '85270919'},
  '2025-07-30': {'1. open': '322.1800',
   '2. high': '324.4499',
   '3. low': '311.6164',
   '4. close': '319.0400',
   '5. volume': '83931942'},
  '2025-07-29': {'1. open': '325.5500',
   '2. high': '326.2500',
   '3. low': '318.2500',
   '4. close': '321.2000',
   '5. volume': '87358861'},
  '2025-07-28': {'1. open': '318.4500',
   '2. high': '330.4900',
   '3. low': '315.6900',
   '4. close': '325.5900',
   '5. volume': '112673755'},
  '2025-07-25': {'1. open': '30

In [43]:
import pandas as pd

cleaned_data = {}

for symbol, raw_data in all_data.items():
    try:
        ts_data = raw_data['Time Series (Daily)']
        df = pd.DataFrame.from_dict(ts_data, orient='index')
        df = df.rename(columns={
            '1. open': 'Open',
            '2. high': 'High',
            '3. low': 'Low',
            '4. close': 'Close',
            '5. volume': 'Volume'
        })
        df.index = pd.to_datetime(df.index)
        df = df.astype(float).sort_index()
        cleaned_data[symbol] = df
    except KeyError:
        print(f"Error fetching or parsing data for {symbol}")

combined_df = pd.concat(cleaned_data.values(), axis=1, keys=cleaned_data.keys())

print(combined_df.head())


               AAPL                                          GOOGL            \
               Open      High     Low   Close      Volume     Open      High   
2025-03-11  223.805  225.8399  217.45  220.84  76137410.0  164.910  166.7500   
2025-03-12  220.140  221.7500  214.91  216.98  62547467.0  166.580  167.6399   
2025-03-13  215.950  216.8394  208.42  209.68  61368330.0  166.035  166.1300   
2025-03-14  211.250  213.9500  209.58  213.49  60107582.0  163.270  166.4900   
2025-03-17  213.310  215.2200  209.97  214.00  48073426.0  165.030  166.3000   

                                        ...     AMZN                      \
               Low   Close      Volume  ...     Open      High       Low   
2025-03-11  161.37  164.04  39587414.0  ...  193.900  200.1800  193.4000   
2025-03-12  163.53  167.11  28372396.0  ...  200.720  201.5200  195.2900   
2025-03-13  162.11  162.76  31756214.0  ...  198.165  198.8799  191.8200   
2025-03-14  162.45  165.49  31995894.0  ...  197.410  198.6

In [44]:
combined_df

AAPL                                           GOOGL            \
               Open      High     Low   Close       Volume     Open      High   
2025-03-11  223.805  225.8399  217.45  220.84   76137410.0  164.910  166.7500   
2025-03-12  220.140  221.7500  214.91  216.98   62547467.0  166.580  167.6399   
2025-03-13  215.950  216.8394  208.42  209.68   61368330.0  166.035  166.1300   
2025-03-14  211.250  213.9500  209.58  213.49   60107582.0  163.270  166.4900   
2025-03-17  213.310  215.2200  209.97  214.00   48073426.0  165.030  166.3000   
...             ...       ...     ...     ...          ...      ...       ...   
2025-07-28  214.030  214.8450  213.06  214.05   37858017.0  193.650  194.0500   
2025-07-29  214.175  214.8100  210.82  211.27   51411723.0  192.430  195.9200   
2025-07-30  211.895  212.3900  207.72  209.05   45512514.0  195.600  197.5999   
2025-07-31  208.490  209.8400  207.16  207.57   80698431.0  195.710  195.9900   
2025-08-01  210.865  213.5800  201.50  202.38  104434473.0  189.025  190.8300   

                                         ...     AMZN                      \
                Low   Close      Volume  ...     Open      High       Low   
2025-03-11  161.370  164.04  39587414.0  ...  193.900  200.1800  193.4000   
2025-03-12  163.530  167.11  28372396.0  ...  200.720  201.5200  195.2900   
2025-03-13  162.110  162.76  31756214.0  ...  198.165  198.8799  191.8200   
2025-03-14  162.450  165.49  31995894.0  ...  197.410  198.6500  195.3200   
2025-03-17  163.670  164.29  31184335.0  ...  198.770  199.0000  194.3247   
...             ...     ...         ...  ...      ...       ...       ...   
2025-07-28  190.840  192.58  38139504.0  ...  233.350  234.2900  232.2500   
2025-07-29  192.080  195.75  41389153.0  ...  234.150  234.7200  230.3100   
2025-07-30  194.685  196.53  32445405.0  ...  231.640  231.8000  229.2900   
2025-07-31  191.090  191.90  51329163.0  ...  235.770  236.5300  231.4000   
2025-08-01  187.820  189.13  34832181.0  ...  217.210  220.4399  212.8000   

                                    TSLA                              \
             Close       Volume     Open      High       Low   Close   
2025-03-11  196.59   54002880.0  225.305  237.0649  217.0200  230.58   
2025-03-12  198.89   43679284.0  247.220  251.8400  241.1000  248.09   
2025-03-13  193.89   41270761.0  248.125  248.2900  232.6000  240.68   
2025-03-14  197.95   38096663.0  247.310  251.5800  240.7300  249.98   
2025-03-17  195.74   47341752.0  245.055  245.4000  232.8000  238.01   
...            ...          ...      ...       ...       ...     ...   
2025-07-28  232.79   26300138.0  318.450  330.4900  315.6900  325.59   
2025-07-29  231.01   33716220.0  325.550  326.2500  318.2500  321.20   
2025-07-30  230.19   32993273.0  322.180  324.4499  311.6164  319.04   
2025-07-31  234.11  104357263.0  319.605  321.3700  306.1000  308.27   
2025-08-01  214.75  122258801.0  306.205  309.3100  297.8200  302.63   

                         
                 Volume  
2025-03-11  174896415.0  
2025-03-12  142215681.0  
2025-03-13  114813525.0  
2025-03-14  100242264.0  
2025-03-17  111900565.0  
...                 ...  
2025-07-28  112673755.0  
2025-07-29   87358861.0  
2025-07-30   83931942.0  
2025-07-31   85270919.0  
2025-08-01   89121446.0  

[100 rows x 25 columns]

In [45]:
combined_df.shape

(100, 25)

In [46]:
combined_df.describe()

AAPL                                                    \
             Open        High         Low       Close        Volume   
count  100.000000  100.000000  100.000000  100.000000  1.000000e+02   
mean   206.235380  209.052718  203.750470  206.366300  5.891664e+07   
std      9.246654    8.103700    9.778488    9.062194  2.607704e+07   
min    171.950000  190.335000  169.210100  172.420000  3.449358e+07   
25%    200.597500  202.555000  198.485000  200.390000  4.420077e+07   
50%    207.790000  209.795000  204.645000  207.695000  5.100780e+07   
75%    212.111250  214.582500  209.887500  212.485000  6.285455e+07   
max    223.805000  225.839900  221.020000  223.890000  1.843959e+08   

            GOOGL                                                   ...  \
             Open       High         Low       Close        Volume  ...   
count  100.000000  100.00000  100.000000  100.000000  1.000000e+02  ...   
mean   168.538750  170.74622  166.253410  168.560900  4.043636e+07  ...   
std     12.186303   11.71734   12.405674   12.151839  1.737974e+07  ...   
min    141.550000  148.94500  140.530000  144.700000  2.094187e+07  ...   
25%    160.418750  161.91750  157.320000  160.002500  2.965279e+07  ...   
50%    166.992500  168.48750  165.210000  166.705000  3.453346e+07  ...   
75%    175.708750  178.06975  174.430000  176.327500  4.582583e+07  ...   
max    197.030000  197.95000  194.685000  196.530000  1.277476e+08  ...   

             AMZN                                                    \
             Open        High         Low       Close        Volume   
count  100.000000  100.000000  100.000000  100.000000  1.000000e+02   
mean   203.953580  206.695366  201.394307  204.194400  4.751967e+07   
std     17.395997   16.157075   17.903235   16.913487  2.202266e+07   
min    162.000000  169.600000  161.380000  167.320000  2.630014e+07   
25%    190.498750  193.667500  187.425000  191.835000  3.336620e+07   
50%    205.407500  206.770000  202.676500  205.645000  4.006503e+07   
75%    217.260000  218.909975  214.252500  217.242500  5.190130e+07   
max    235.770000  236.530000  232.250000  234.110000  1.231594e+08   

             TSLA                                                    
             Open        High         Low       Close        Volume  
count  100.000000  100.000000  100.000000  100.000000  1.000000e+02  
mean   296.116250  303.928305  288.879233  296.993900  1.183011e+08  
std     38.814429   37.502383   39.149947   37.605521  3.784469e+07  
min    223.780000  230.100000  214.250000  221.860000  5.804230e+07  
25%    259.590000  276.250000  255.065000  268.165000  9.083359e+07  
50%    303.127500  309.990000  296.085000  303.965000  1.116891e+08  
75%    325.685000  332.410000  318.787500  326.175000  1.369937e+08  
max    365.290000  367.710000  356.000000  362.890000  2.928187e+08  

[8 rows x 25 columns]

In [47]:
combined_df.isnull().sum()

AAPL   Open      0
       High      0
       Low       0
       Close     0
       Volume    0
GOOGL  Open      0
       High      0
       Low       0
       Close     0
       Volume    0
MSFT   Open      0
       High      0
       Low       0
       Close     0
       Volume    0
AMZN   Open      0
       High      0
       Low       0
       Close     0
       Volume    0
TSLA   Open      0
       High      0
       Low       0
       Close     0
       Volume    0
dtype: int64

In [48]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2025-03-11 to 2025-08-01
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   (AAPL, Open)     100 non-null    float64
 1   (AAPL, High)     100 non-null    float64
 2   (AAPL, Low)      100 non-null    float64
 3   (AAPL, Close)    100 non-null    float64
 4   (AAPL, Volume)   100 non-null    float64
 5   (GOOGL, Open)    100 non-null    float64
 6   (GOOGL, High)    100 non-null    float64
 7   (GOOGL, Low)     100 non-null    float64
 8   (GOOGL, Close)   100 non-null    float64
 9   (GOOGL, Volume)  100 non-null    float64
 10  (MSFT, Open)     100 non-null    float64
 11  (MSFT, High)     100 non-null    float64
 12  (MSFT, Low)      100 non-null    float64
 13  (MSFT, Close)    100 non-null    float64
 14  (MSFT, Volume)   100 non-null    float64
 15  (AMZN, Open)     100 non-null    float64
 16  (AMZN, High)     100 non-null    float64
 1